<a href="https://colab.research.google.com/github/Alanvnorcott/CatsAndDogs/blob/main/ClassifyCatAndDogs_Alan_Norcott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os


In [ ]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3


In [ ]:
filenames=os.listdir("/content/drive/MyDrive/2024_ai_cats_dogs_dataset/training_set")
categories=[]
for f_name in filenames:
  category=f_name.split('.')[0]
  if category=='dog':
    categories.append(1)
  else:
    categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
    })


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
Dropout,Flatten,Dense,Activation,\
BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        2

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
rescale=1./255,
shear_range=0.1,
zoom_range=0.2,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1
)


train_generator = train_datagen.flow_from_directory(
    directory="/content/drive/MyDrive/2024_ai_cats_dogs_dataset/training_set",
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size,
    classes=['dogs', 'cats']  # Specify the order of classes
)


validation_datagen = ImageDataGenerator(rescale=1./255)




validation_generator = validation_datagen.flow_from_directory(
    directory="/content/drive/MyDrive/2024_ai_cats_dogs_dataset/training_set",
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size,
    classes=['dogs', 'cats']  # Specify the order of classes
)




test_datagen = ImageDataGenerator(rotation_range=15,
rescale=1./255,
shear_range=0.1,
zoom_range=0.2,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1)


test_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="/content/drive/MyDrive/2024_ai_cats_dogs_dataset/test_set",
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)


Found 5328 images belonging to 2 classes.
Found 5328 images belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
total_train = len(train_generator.filenames)
total_validate = len(validation_generator.filenames)

epochs=10
history = model.fit_generator(
train_generator,
epochs=epochs,
validation_data=validation_generator,
validation_steps=total_validate/batch_size,
steps_per_epoch=total_train/batch_size,
callbacks=callbacks
)


<ipython-input-10-31bd5596bca7>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
356/355 [==============================] - ETA: -3s - loss: 0.8636 - accuracy: 0.6565

355/355 [==============================] - 1407s 4s/step - loss: 0.8636 - accuracy: 0.6565 - val_loss: 0.6722 - val_accuracy: 0.7547 - lr: 0.0010
Epoch 2/10
356/355 [==============================] - ETA: 0s - loss: 0.6000 - accuracy: 0.7292

355/355 [==============================] - 76s 214ms/step - loss: 0.6000 - accuracy: 0.7292 - val_loss: 0.5225 - val_accuracy: 0.7459 - lr: 0.0010
Epoch 3/10
356/355 [==============================] - ETA: 0s - loss: 0.5405 - accuracy: 0.7438

355/355 [==============================] - 75s 212ms/step - loss: 0.5405 - accuracy: 0.7438 - val_loss: 0.4830 - val_accuracy: 0.7757 - lr: 0.0010
Epoch 4/10
356/355 [==============================] - ETA: 0s - loss: 0.5209 - accuracy: 0.7547

...

355/355 [==============================] - 76s 212ms/step - loss: 0.4937 - accuracy: 0.7712 - val_loss: 0.4306 - val_accuracy: 0.8076 - lr: 0.0010
Epoch 7/10
356/355 [==============================] - ETA: 0s - loss: 0.4684 - accuracy: 0.7838

355/355 [==============================] - 92s 258ms/step - loss: 0.4684 - accuracy: 0.7838 - val_loss: 0.5101 - val_accuracy: 0.7660 - lr: 0.0010
Epoch 8/10
356/355 [==============================] - ETA: 0s - loss: 0.4485 - accuracy: 0.8020

355/355 [==============================] - 76s 213ms/step - loss: 0.4485 - accuracy: 0.8020 - val_loss: 0.7140 - val_accuracy: 0.6503 - lr: 0.0010
Epoch 9/10
356/355 [==============================] - ETA: 0s - loss: 0.4348 - accuracy: 0.7975

355/355 [==============================] - 91s 257ms/step - loss: 0.4348 - accuracy: 0.7975 - val_loss: 0.4189 - val_accuracy: 0.8133 - lr: 0.0010
Epoch 10/10
356/355 [==============================] - ETA: 0s - loss: 0.4387 - accuracy: 0.7973

355/355 [==============================] - 73s 207ms/step - loss: 0.4387 - accuracy: 0.7973 - val_loss: 0.4872 - val_accuracy: 0.8065 - lr: 0.0010


In [ ]:
model.save("model1_catsVSdogs_10epoch.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
test_filenames = os.listdir("./dogs-vs-cats/test1")
test_df = pd.DataFrame({
'filename': test_filenames
})
nb_samples = test_df.shape[0]

FileNotFoundError: [Errno 2] No such file or directory: './dogs-vs-cats/test1'

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
filename = row['filename']
category = row['category']
img = load_img("./dogs-vs-cats/test1/"+filename, target_size=Image_Size)
plt.subplot(6, 3, index+1)
plt.imshow(img)
plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

In [ ]:
results={
0:'cat',
1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("__image_path_TO_custom_image")
im=im.resize(Image_Size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred=model.predict_classes([im])[0]
print(pred,results[pred])

In [ ]:
from PIL import Image
import numpy as np
from keras.models import load_model


# Load the trained model
model = load_model('model1_catsVSdogs_10epoch.h5')


# Dictionary to label all traffic signs class.
classes = {
    0: 'its a cat',
    1: 'its a dog',
}


def classify_image(file_path):
    image = Image.open(file_path)
    image = image.resize((128, 128))
    image = np.expand_dims(np.array(image), axis=0)
    image = image / 255.0
    pred = model.predict_classes([image])[0]
    sign = classes[pred]
    return sign


# Specify the path to the image you want to classify
image_path = 'path/to/your/image.jpg'


# Call the classify_image function with the image path
classification_result = classify_image(image_path)
print(classification_result)